## Procesamiento inicial del audio

En este caso, lo que se va a hacer es trasformar el audio a **.mp3**, para luego dividir el audio a secciones más pequeñas. Para ello, al igual que en el resto de pruebas, se usará la librería de [pydub](https://pydub.com)

también he tomado como referencia para el tema de la división de los audios [medium](https://medium.com/@purushottamgupta2010/splitting-audio-files-into-manageable-segments-with-python-890aba53d254)

In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pydub

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

os.path.exists("./audios/audio_prueba.ogg")

True

In [2]:
from pydub import AudioSegment
import math

audio = AudioSegment.from_file("./audios/audio_prueba.ogg")

base_length = 15 * 60 * 1000 # Longitud de tiempo base que van a tener los audios
total_length = len(audio) # Recuerda que el tiempo que te va a poner por aquí es el tiempo en milisegundos

num_segments = math.ceil(total_length / base_length)
new_length = total_length / num_segments

files = []

for i in range(num_segments):
    start = i * new_length
    finish = min(total_length, ((i + 1 ) * new_length))
    
    if i == 0:
        finish = min(total_length, finish + 2 * 1000)
    else:
        start = max(0, start - 2 * 1000)

    finish = min(finish, total_length)
    
    segment = audio[start:finish]
    
    output_file = f"./audios/salida/audio_prueba_part{i+1}.mp3"
    files.append(output_file)

    segment.export(output_file, format="mp3", bitrate="128k")

    print(f"Exportado: {output_file}")

Exportado: ./audios/salida/audio_prueba_part1.mp3
Exportado: ./audios/salida/audio_prueba_part2.mp3
Exportado: ./audios/salida/audio_prueba_part3.mp3
Exportado: ./audios/salida/audio_prueba_part4.mp3
Exportado: ./audios/salida/audio_prueba_part5.mp3
Exportado: ./audios/salida/audio_prueba_part6.mp3
Exportado: ./audios/salida/audio_prueba_part7.mp3
Exportado: ./audios/salida/audio_prueba_part8.mp3
Exportado: ./audios/salida/audio_prueba_part9.mp3
Exportado: ./audios/salida/audio_prueba_part10.mp3
Exportado: ./audios/salida/audio_prueba_part11.mp3
Exportado: ./audios/salida/audio_prueba_part12.mp3
Exportado: ./audios/salida/audio_prueba_part13.mp3
Exportado: ./audios/salida/audio_prueba_part14.mp3
Exportado: ./audios/salida/audio_prueba_part15.mp3
Exportado: ./audios/salida/audio_prueba_part16.mp3
Exportado: ./audios/salida/audio_prueba_part17.mp3
Exportado: ./audios/salida/audio_prueba_part18.mp3
Exportado: ./audios/salida/audio_prueba_part19.mp3


## Uso del modelo

In [2]:
pip install --upgrade transformers datasets[audio] accelerate

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

result = pipe(files, batch_size=2, return_timestamps=True)

In [4]:
import torch
torch.cuda.empty_cache() #Limpieza de la VRAM

In [ ]:
result

In [6]:
with open("./salida/salida_whisper_large_varios_audios(3).txt", "w") as archivo:
    for segmento in result:
        archivo.write(segmento['text'])

## Resultados de las pruebas:

